In [ ]:
# PACKAGE IMPORTS #
from pprint import pprint
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import csv
import json
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import matplotlib.pyplot as plt
%matplotlib inline

import bokeh

### Create dataframe object from data in local file (.csv)

In [ ]:
def load_and_preprocess_data(filename, emails_req=True):
    """Load data from file and apply transformations"""
    #Load data from csv
    df = pd.read_csv(filename)
    #pre-process data cleaning
    df['date'] = pd.to_datetime(df['date']) # convert to datetime object
    df['week_num'] = df['date'].dt.week # get WEEK number
    df['month_num'] = df['date'].dt.month # get MONTH number
    df = df.dropna(subset=['email']) # remove records with NO email address
    #filter to relevant columns
    col_list = [
            'user_distinct_id', 
            'email', 
            'date',
            'week_num',
            'month_num',
            'python_edit',
            'completions_num_selected',
            'num_events_with_python'
        ]
    df = df[col_list]
    return df

In [ ]:
df_ks1d_all = load_and_preprocess_data('Completions_Analysis_ks1d.csv')
print(df_ks1d_all.columns.tolist())

In [ ]:
df_ks1d_all.isnull().sum() # check for NULL values

In [ ]:
df_ks1d_all.shape

In [ ]:
df = pd.read_csv('Completions_Analysis_ks1d.csv')

In [ ]:
df.shape

### Analyze kite_status_1d events in Dataframe

In [ ]:
#dataframe for ks1d events including all results from JQL query
df = df_ks1d_all.copy()

#working dataframe object for exploration: df
df.head()

In [ ]:
# example of how to create new dataframe objects that contain only desired rows, cols
# df_recent = df.loc[df['date'] > '2020-04-01']

#df_recent.head()

In [ ]:
#df.loc[df['email'] == 'caelan@kite.com'].sort_values('python_edit', ascending=False)

In [ ]:
# Example of computing new stats and append a new column to the dataframe
#df['ratio_compl_triggered_to_pyedit'] = df.completions_triggered / df.python_edit
#df.head()

### Python_edit total for a prior calendar period

In [ ]:
#df.loc[:,['date', 'python_edit']][df.date < '2019-10-21']

In [ ]:
# use datetime library for relative time operations
now = datetime.now()
one_week = timedelta(days=7)

one_week_ago = now - one_week - timedelta(days=0) # use last member in equation to adjust date
print(one_week_ago.month, one_week_ago.day)

In [ ]:
# Dataframe with events after a specified date
# df_lastweek = df.loc[:,['email','date', 'python_edit']][df.date > '2019-10-21']

# Dataframe with events from this calendar year-to-date

df_ytd = df.loc[:,['email','date', 'python_edit']][df.date >= '2020-01-01']

### Counts by week per user (email)

In [ ]:
df_weekly_totals = df.groupby(['email', 'week_num']).sum()
df_weekly_totals # num_events_with_python

### Counts by month per user (email)

In [ ]:
df_monthly_totals = df.groupby(['email', 'month_num']).sum()
df_monthly_totals # num_events_with_python

### Defining last week and month

In [ ]:
# WEEK LOGIC
# one_week = timedelta(days=7)
# one_week_ago = datetime.now() - one_week
last_full_week_num = one_week_ago.isocalendar()[1] 

# MONTH LOGIC
# one_month = timedelta(month = 1)
last_full_month_num = datetime.now().month - 1

# Note: ISO calendar weeks begin Monday, end Sunday

# Check dates:
print("One week ago:", (one_week_ago))
print(last_full_week_num)

print("One month ago:", last_full_month_num)

### Counts for last week per user (email)

In [ ]:
# groupby week_num table
df_last_week = df[df['week_num'] == last_full_week_num] \
  .groupby(['email', 'week_num']).sum().drop(['month_num'], axis=1)

df_last_week

In [ ]:
# Weekly: Only include users who had positive weekly completions selected totals 
df_last_week = df_last_week[df_last_week.completions_num_selected > 0]
df_last_week

### Counts for last month per user (email)

In [ ]:
# groupby month_num table
df_last_month = df[df['month_num'] == last_full_month_num] \
  .groupby(['email', 'month_num']).sum().drop(['week_num'], axis=1)

df_last_month

In [ ]:
# Monthly: Only include users who had positive monthly completions selected totals 
df_last_month = df_last_month[df_last_month.completions_num_selected > 0]
df_last_month

### Counts year-to-date per user (email) 

In [ ]:
df_ytd_totals = df.groupby(['email']).sum().drop(['week_num', 'month_num'], axis=1)
df_ytd_totals.columns = [str(col) + '_ytd' for col in df_ytd_totals.columns] #rename columns to include ytd
df_ytd_totals

### Combine Weekly with YTD and export to CSV

In [ ]:
combine_with_weekly_df = pd.merge(df_last_week, df_ytd_totals, how='inner', on='email')

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
print(time_stamp)

combine_with_weekly_df.to_csv('completion_stats_per_user_last_week_' + time_stamp + '.csv') # Note: Date of export in the filename
combine_with_weekly_df

### Combine Monthly with YTD and export to CSV

In [ ]:
combine_with_monthly_df = pd.merge(df_last_month, df_ytd_totals, how='inner', on='email')

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
print(time_stamp)

combine_with_monthly_df.to_csv('completion_stats_per_user_last_month_' + time_stamp + '.csv') # Note: Date of export in the filename
combine_with_monthly_df

### Check final joined table by searching for a user (email)

In [ ]:
# For spot-checking data and individual queries
individual_email = 'alexander.nilsson@evado.se'
indiv_stats = combine_with_monthly_df[combine_with_monthly_df.index == individual_email]

print(indiv_stats)

### Data Import Option: Get JQL query results directly from Mixpanel API (requires config)

In [ ]:
#from mixpanel_jql import JQL, Reducer, Events

#mixpanel_api_secret = '...'

# query = JQL(
#             mixpanel_api_secret,
#             events=Events({
#                 'event_selectors': [{'event': "X"}],
#                 'from_date': datetime(2016, 5, 1),
#                 'to_date': datetime(2016, 5, 31)
#             })
#         ).group_by(
#             keys=[
#                 "new Date(e.time).toISOString().split('T')[0]",
#             ],
#             accumulator=Reducer.count()
#         )

### Build dataframe from mixpanel-jql query which streams back results
#for row in query.send():
#  df